In [518]:
import matplotlib.pyplot as plt
import plotly.express as px
import utils
import importlib
import matplotlib.dates as mdates
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.ticker as plticker
importlib.reload(utils)
import datetime as dt
import numpy as np
import warnings

import pandas as pd
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

* load logs
* pick trips/events
* x-axis are time
* y-axis # of passengers picked up
* text on top of line is explanation or overload bus id

In [519]:
# Load logs
log_file = 'logs/all_day_nobreak/no_inject_20210713_BL_nobreak.log'
log_file = 'logs/all_day_nobreak/no_inject_20210713_ALL_IT200_nobreak.log'
# log_file = 'logs/no_inject_mcts.log'
# log_file = 'logs/no_inject_20210507_BL.log'
# log_file = 'logs/no_inject_20210607_all_served_IT200.log'
df = utils.get_bus_df(log_file)
df.got_on_bus.sum(), df.remain.sum()

(98226, 83178)

In [520]:
trip_ids = df.trip_id.unique().tolist()

In [521]:
log_files = ['logs/all_day_nobreak/no_inject_20210713_BL_nobreak.log',
             'logs/all_day_nobreak/no_inject_20210713_ALL_IT200_nobreak.log']
df_arr = []
dispatch_arr = []
for log_file in log_files:
    df = utils.get_bus_df(log_file)
    dispatch_df = utils.get_dispatch_df(log_file)
    df_arr.append(df)
    dispatch_arr.append(dispatch_df)
    
for trip_id in trip_ids:
    for df in df_arr:
        gob = df.query("trip_id == '245771'").sort_values('actual_time').got_on_bus.sum()
        rem = df.query("trip_id == '245771'").sort_values('actual_time').remain.sum()


In [522]:
a = df_arr[0].groupby('trip_id').agg({'bus_id':'first', 'actual_time':'first', 'got_on_bus':'sum', 'remain':'sum'}).rename({'got_on_bus':'bl_gob', 'remain':'bl_rem'}, axis=1)
b = df_arr[1].groupby('trip_id').agg({'bus_id':'first', 'actual_time':'first', 'got_on_bus':'sum', 'remain':'sum'}).rename({'got_on_bus':'mcts_gob', 'remain':'mcts_rem'}, axis=1)
c = a.merge(b, on=['trip_id', 'bus_id', 'actual_time'])
c[c['mcts_gob'] > c['bl_gob']]

,bus_id,actual_time,bl_gob,bl_rem,mcts_gob,mcts_rem
trip_id,,,,,,
241618,2013,2021-07-13 16:15:00,120,228,178,228
241620,2013,2021-07-13 18:15:00,133,246,300,325
241623,2013,2021-07-13 21:15:00,257,263,313,306
241634,2013,2021-07-13 09:37:00,78,12,90,12
241641,2013,2021-07-13 16:43:00,102,237,244,332
...,...,...,...,...,...,...
245920,1702,2021-07-13 22:15:00,95,49,103,17
245926,1702,2021-07-13 20:55:00,121,53,174,53
245927,1827,2021-07-13 21:35:00,167,302,340,407


In [523]:
dispatch_arr[0].set_index('actual_time').between_time('16:00', '16:43')

,bus_id,from_stop,trip_id,stop_id
actual_time,,,,
2021-07-13 16:00:00,45,BORHOSP,244835,MXOMTVIE
2021-07-13 16:09:04,43,GXIRIVEF,241640,UNI2AWN
2021-07-13 16:24:00,41,21WE,245698,21AEDGNF
2021-07-13 16:26:14,42,MCSHERM,244837,MXOMTVIE
2021-07-13 16:27:43,43,MCC4_20,244917,MURMCGWN
2021-07-13 16:27:45,45,HICHICNN,241937,MCC5_12
2021-07-13 16:35:00,41,MCC5_9,244990,MCC4_16
2021-07-13 16:38:00,44,DWMRT,242330,FAT19SEN


In [524]:
df_arr[0].query("trip_id == '241641'").sort_values('actual_time')

,actual_time,bus_id,trip_id,scheduled_time,stop_id,got_on_bus,waiting_to_board,offs,remain,load,overload_id
83032,2021-07-13 16:43:00,2013,241641,2021-07-13 16:43:00,BOYBUENN,10,49,0,39,10,0
83429,2021-07-13 16:45:50,2013,241641,2021-07-13 16:45:50,BUEWHASN,0,29,0,29,10,0
83488,2021-07-13 16:46:12,2013,241641,2021-07-13 16:46:12,ROWBUENN,0,5,0,5,10,0
83533,2021-07-13 16:46:34,2013,241641,2021-07-13 16:46:34,ROWCRONN,0,0,10,0,0,0
83622,2021-07-13 16:47:07,2013,241641,2021-07-13 16:47:07,ROWBONEF,6,6,0,0,6,0
83730,2021-07-13 16:47:47,2013,241641,2021-07-13 16:47:47,ROWCRAEN,0,0,6,0,0,0
83806,2021-07-13 16:48:22,2013,241641,2021-07-13 16:48:22,REVROWEF,0,0,0,0,0,0
83875,2021-07-13 16:48:46,2013,241641,2021-07-13 16:48:46,REVPAINS,10,10,0,0,10,0
83965,2021-07-13 16:49:16,2013,241641,2021-07-13 16:49:16,REVWHIWN,0,18,0,18,10,0
84092,2021-07-13 16:50:00,2013,241641,2021-07-13 16:50:00,WHIMOOSN,0,0,7,0,3,0


In [525]:
dispatch_arr[1].set_index('actual_time').between_time('16:00', '16:44')

,bus_id,from_stop,trip_id,stop_id
actual_time,,,,
2021-07-13 16:05:00,45,MCC5_1,241936,MCC5_12
2021-07-13 16:07:29,43,GXIRIVEF,245083,GXIRIVEF
2021-07-13 16:12:17,41,DWMRT,243423,21ASCASM
2021-07-13 16:12:38,44,TEN51S,245664,BRO9AWF
2021-07-13 16:16:32,45,BORHOSP,245874,MCC4_24
2021-07-13 16:23:39,44,CHI23AWF,242330,MCGSTRWN
2021-07-13 16:24:00,41,21WE,242311,UNI2AEF
2021-07-13 16:38:00,42,DWMRT,245827,DICMAPSN
2021-07-13 16:42:56,43,MCC4_16,241949,MCC5_12


In [526]:
import pandas as pd
with pd.option_context('display.max_rows', 10,):
    display(df_arr[1].query("trip_id == '241641'").sort_values('actual_time'))

,actual_time,bus_id,trip_id,scheduled_time,stop_id,got_on_bus,waiting_to_board,offs,remain,load,overload_id
86663,2021-07-13 16:43:00,2013,241641,2021-07-13 16:43:00,BOYBUENN,10,49,0,39,10,0
87100,2021-07-13 16:45:50,2013,241641,2021-07-13 16:45:50,BUEWHASN,0,29,0,29,10,0
87161,2021-07-13 16:46:12,2013,241641,2021-07-13 16:46:12,ROWBUENN,0,5,0,5,10,0
87208,2021-07-13 16:46:34,2013,241641,2021-07-13 16:46:34,ROWCRONN,0,0,10,0,0,0
87302,2021-07-13 16:47:07,2013,241641,2021-07-13 16:47:07,ROWBONEF,6,6,0,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...
91637,2021-07-13 17:20:40,42,241641,2021-07-13 17:02:31,1SSPRSM,0,0,4,0,45,1
91764,2021-07-13 17:21:47,42,241641,2021-07-13 17:03:34,1SOLDSM,5,5,0,0,50,1
91856,2021-07-13 17:22:28,42,241641,2021-07-13 17:05:09,WOON1SWF,0,0,8,0,42,1
91956,2021-07-13 17:23:22,42,241641,2021-07-13 17:06:50,UNI2AWN,0,0,2,0,40,1


In [527]:
trip_ids = ['244837', '244917', '241937', '244990', '244990'] + ['242311', '245827', '241949', '242330', '245874'] + ['241641']
df_arr[0][df_arr[0].trip_id.isin(trip_ids)].remain.sum(), df_arr[1][df_arr[1].trip_id.isin(trip_ids)].remain.sum()

(2722, 2857)

In [528]:
def merge_all_indices(valid_df, all_indices):
    dt_indices = np.unique(np.asarray(all_indices))
    dt_indices = pd.to_datetime(dt_indices) 
    valid_df = valid_df.set_index('actual_time')
    a = pd.DataFrame(index=dt_indices)
    all_indices = valid_df.join(a, how='right').fillna(0)
    return all_indices

In [529]:
dt_indices = df.actual_time.unique().tolist() + dispatch_df.actual_time.unique().tolist()

df_index = 0
# tdf = df_arr[df_index][df_arr[df_index].overload_id == 0]
tdf = df_arr[df_index]
tdf = tdf[tdf['trip_id'].isin(trip_ids)]
all_indices = merge_all_indices(tdf, dt_indices)

only_valid = all_indices[all_indices['trip_id'].isin(trip_ids)]
min_time = (only_valid.index.min() - dt.timedelta(minutes=5)).time()
max_time = (only_valid.index.max() - dt.timedelta(minutes=5)).time()
merged_df = all_indices.between_time(min_time, max_time)
merged_df['zero'] = 0
merged_df['remain'] = merged_df['remain'] * -1

In [530]:
merged_df.head()

,bus_id,trip_id,scheduled_time,stop_id,got_on_bus,waiting_to_board,offs,remain,load,overload_id,zero
2021-07-13 15:20:00,0,0,0,0,0.0,0.0,0.0,-0.0,0.0,0.0,0
2021-07-13 15:20:01,0,0,0,0,0.0,0.0,0.0,-0.0,0.0,0.0,0
2021-07-13 15:20:02,0,0,0,0,0.0,0.0,0.0,-0.0,0.0,0.0,0
2021-07-13 15:20:03,0,0,0,0,0.0,0.0,0.0,-0.0,0.0,0.0,0
2021-07-13 15:20:05,0,0,0,0,0.0,0.0,0.0,-0.0,0.0,0.0,0


In [531]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=merged_df.index, y=merged_df.zero, line=dict(color='black', width=1), showlegend=False))

# marker_pattern_shape=".", marker_pattern_fillmode='replace'
traces = [go.Bar(x=trip_df.index, y=trip_df.got_on_bus, width=25000, name="regular bus<br>passenger served", marker_color='grey', marker_opacity=0.5, showlegend=i==0) \
            for i, (trip_id, trip_df) in enumerate(merged_df.query("trip_id != 0 and overload_id == 0").groupby("trip_id"))]
fig.add_traces(traces)

fig.add_vrect(x0=merged_df.index.min(), x1=merged_df.index.max(), 
          annotation_text=f"{int(merged_df.got_on_bus.sum())}", 
          annotation_position="top left", annotation_textangle=0, annotation=dict(font=dict(size=20), bgcolor="rgba(255,255,255,0.8)"),
          fillcolor="green", opacity=0, line_width=0)

# TODO: Change these to people who left, not remaining
# traces = [go.Bar(x=trip_df.index, y=trip_df.remain, width=25000, name=trip_id, marker_color='grey', marker_opacity=0.5) \
#             for trip_id, trip_df in merged_df.query("trip_id != 0 and overload_id == 0").groupby("trip_id")]
# fig.add_traces(traces)

marker_colors = ['blue', 'blue']
traces = [go.Bar(x=trip_df.index, y=trip_df.got_on_bus, width=35000, name="overflow bus<br>passengers served", marker_color=marker_colors[i%2], showlegend=i==0) \
            for i, (trip_id, trip_df) in enumerate(merged_df.query("trip_id != 0 and overload_id == 1").groupby("trip_id"))]
fig.add_traces(traces)

# traces = [go.Bar(x=trip_df.index, y=trip_df.remain, width=55000, name=trip_id, marker_color='red') \
#             for trip_id, trip_df in merged_df.query("trip_id != 0 and overload_id == 1").groupby("trip_id")]
# fig.add_traces(traces)

dispatch_df = dispatch_arr[df_index]
dispatch_df = dispatch_df[dispatch_df['trip_id'].isin(trip_ids)]
all_indices = merge_all_indices(dispatch_df, dt_indices).query("bus_id != 0")
    
for i, (k, v) in enumerate(all_indices.sort_index().iterrows()):
  

    fig.add_trace(go.Scatter(x=[k,k], 
                        y=[0, 0], 
                        mode='lines', 
                        line=dict(color='black', dash='dash'),
                        name='overflow<br>dispatched', showlegend=i==0))
    
    trip_id = v['trip_id']
    bus_id = v['bus_id']
    fig.add_vline(x=k, line_width=2, line_dash="dash", line_color="black")
    a = merged_df.query(f"bus_id == '{bus_id}' and trip_id == '{trip_id}'")
    a_start = a.index.min()
    a_end = a.index.max()
    a_sum_served = int(a.got_on_bus.sum())
    # a_start, a_end, a_sum_served
    if i % 2 == 0:
      shift = -80
    else:
      shift = 0
    print(a_sum_served, i, shift)
    fig.add_vrect(x0=k, x1=a_end, 
            #   annotation_text=f"Total served:{a_sum_served}", annotation_position="top left",
              annotation_text=f"{bus_id}:{a_sum_served}", annotation_position="top left", annotation_textangle=-90, annotation=dict(font=dict(size=20), yshift=shift, bgcolor="rgba(255,255,255,0.8)"),
              fillcolor="green", opacity=0, line_width=0)

# fig.update_layout(showlegend=False)
fig.update_layout(margin={"r":10,"t":10,"l":10,"b":10}, width=1200, height=300)
fig.update_yaxes(range=[-40, 40], title='passengers served')
fig.update_yaxes(range=[0, 40])

# fig.write_image("plots/comparison.eps")
fig.show()

0 0 -80
27 1 0
111 2 -80
138 3 0
44 4 -80


In [532]:
27 + 111 + 138 + 44

320

In [533]:
193 + 50 + 8 + 109+ 104 

464

# SUBPLOTS

In [534]:
log_files = ['logs/all_day_nobreak/no_inject_20210713_BL_nobreak.log',
             'logs/all_day_nobreak/no_inject_20210713_ALL_IT200_nobreak.log']
df_arr = []
dispatch_arr = []
for log_file in log_files:
    df = utils.get_bus_df(log_file)
    dispatch_df = utils.get_dispatch_df(log_file)
    df_arr.append(df)
    dispatch_arr.append(dispatch_df)
    
for trip_id in trip_ids:
    for df in df_arr:
        gob = df.query("trip_id == '245771'").sort_values('actual_time').got_on_bus.sum()
        rem = df.query("trip_id == '245771'").sort_values('actual_time').remain.sum()
        

trip_ids = ['244837', '244917', '241937', '244990', '244990'] + ['242311', '245827', '241949', '242330', '245874'] + ['241641']

In [535]:
dt_indices = df.actual_time.unique().tolist() + dispatch_df.actual_time.unique().tolist()

In [536]:
fig = make_subplots(
    rows=2, cols=1,
    horizontal_spacing = 0.05, vertical_spacing=0.1
)

for df_index in [0, 1]:
  tdf = df_arr[df_index]
  tdf = tdf[tdf['trip_id'].isin(trip_ids)]
  all_indices = merge_all_indices(tdf, dt_indices)

  only_valid = all_indices[all_indices['trip_id'].isin(trip_ids)]
  min_time = (only_valid.index.min() - dt.timedelta(minutes=5)).time()
  max_time = (only_valid.index.max() - dt.timedelta(minutes=5)).time()
  merged_df = all_indices.between_time(min_time, max_time)
  merged_df['zero'] = 0
  merged_df['remain'] = merged_df['remain'] * -1
  
  fig.add_trace(go.Scatter(x=merged_df.index, y=merged_df.zero, line=dict(color='black', width=1), showlegend=False), row=df_index+1, col=1)

  # marker_pattern_shape=".", marker_pattern_fillmode='replace'
  traces = [go.Bar(x=trip_df.index, y=trip_df.got_on_bus, width=25000, name="regular bus passenger served", marker_color='grey', marker_opacity=0.5, showlegend=(i==0 and df_index == 0)) \
              for i, (trip_id, trip_df) in enumerate(merged_df.query("trip_id != 0 and overload_id == 0").groupby("trip_id"))]
  fig.add_traces(traces, rows=df_index+1, cols=1)

  fig.add_vrect(x0=merged_df.index.min(), x1=merged_df.index.max(), 
            annotation_text=f"Total served:{int(merged_df.got_on_bus.sum())}", 
            annotation_position="top left", annotation_textangle=0, annotation=dict(font=dict(size=20), bgcolor="rgba(255,255,255,0.8)"),
            fillcolor="green", opacity=0, line_width=0, row=df_index+1, col=1)
  
  marker_colors = ['blue', 'blue']
  traces = [go.Bar(x=trip_df.index, y=trip_df.got_on_bus, width=35000, name="overflow bus passengers served", marker_color=marker_colors[i%2], showlegend=(i==0 and df_index == 0)) \
              for i, (trip_id, trip_df) in enumerate(merged_df.query("trip_id != 0 and overload_id == 1").groupby("trip_id"))]
  fig.add_traces(traces, rows=df_index+1, cols=1)

  dispatch_df = dispatch_arr[df_index]
  dispatch_df = dispatch_df[dispatch_df['trip_id'].isin(trip_ids)]
  all_indices = merge_all_indices(dispatch_df, dt_indices).query("bus_id != 0")
      
  for i, (k, v) in enumerate(all_indices.sort_index().iterrows()):
    

      fig.add_trace(go.Scatter(x=[k,k], 
                          y=[0, 0], 
                          mode='lines', 
                          line=dict(color='black', dash='dash'),
                          name='overflow dispatched', showlegend=(i==0 and df_index == 0)), row=df_index+1, col=1)
      
      trip_id = v['trip_id']
      bus_id = v['bus_id']
      fig.add_vline(x=k, line_width=2, line_dash="dash", line_color="black", row=df_index+1, col=1)
      a = merged_df.query(f"bus_id == '{bus_id}' and trip_id == '{trip_id}'")
      a_start = a.index.min()
      a_end = a.index.max()
      a_sum_served = int(a.got_on_bus.sum())
      # a_start, a_end, a_sum_served
      if i % 2 == 0:
        shift = -80
      else:
        shift = 0
      # print(a_sum_served, i, shift)
      fig.add_vrect(x0=k, x1=a_end, 
              #   annotation_text=f"Total served:{a_sum_served}", annotation_position="top left",
                annotation_text=f"{bus_id}:{a_sum_served}", annotation_position="top left", annotation_textangle=-90, annotation=dict(font=dict(size=20), yshift=shift, bgcolor="rgba(255,255,255,0.8)"),
                fillcolor="green", opacity=0, line_width=0, row=df_index+1, col=1)

# fig.update_layout(showlegend=False)
fig.update_layout(margin={"r":10,"t":10,"l":10,"b":10}, width=1400, height=600)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
    font=dict(size=15)
))
fig.update_yaxes(range=[-40, 40], title='passengers served')
fig.update_yaxes(range=[0, 40])

fig.write_image("plots/timeseries_comparison.eps")
fig.show()

## BREAKDOWN HANDLING

In [537]:
log_files = ['logs/all_day_nobreak/no_inject_20210607_BL_nobreak.log',
             'logs/all_day_break_issue/no_inject_20210607_BL.log',
             'logs/all_day_break_issue/no_inject_20210607_all_served_IT200.log']
df_arr = []
dispatch_arr = []
for log_file in log_files:
    df = utils.get_bus_df(log_file)
    # dispatch_df = utils.get_dispatch_df(log_file)
    df_arr.append(df)
    # dispatch_arr.append(dispatch_df)
    
# for trip_id in trip_ids:
#     for df in df_arr:
#         gob = df.query("trip_id == '245771'").sort_values('actual_time').got_on_bus.sum()
#         rem = df.query("trip_id == '245771'").sort_values('actual_time').remain.sum()

In [538]:
import re
def get_dispatch_df(log_file):
    # Dispatching overflow bus 44 from BNA @ stop 8ABROSN
    DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'
    lines_before = []
    bus_stats = []
    with open(log_file) as file:
        for line in file:
            lines_before.append(line)
            if 'Dispatching overflow' in line:
                dispatch_info = lines_before[-3].rstrip()
                data = line.rstrip()
                in_brackets = re.findall("\[(.*?)\]", data)
                out_brackets = re.findall(r"(.*?)(?:\[.*?\]|$)", data)
                log_text = out_brackets[2].strip().split(" ")
                
                time = dt.datetime.strptime(in_brackets[1], DATETIME_FORMAT)
                overflow_bus_id = log_text[3]
                from_stop = log_text[5]
                dispatch_stop = log_text[-1]
                
                dispatch_trip = dispatch_info.split(" ")[-1].split("'")[1]
                # print(time, overflow_bus_id, from_stop, dispatch_stop)
                
                entry = {"actual_time":time,
                        "bus_id":overflow_bus_id,
                        "from_stop":from_stop,
                        "trip_id":dispatch_trip,
                        "stop_id":dispatch_stop,
                        "type":"overflow"}
                bus_stats.append(entry)
                lines_before = []
            
            if 'Sending takeover overflow bus' in line:
                dispatch_info = lines_before[-3].rstrip()
                data = line.rstrip()
                in_brackets = re.findall("\[(.*?)\]", data)
                out_brackets = re.findall(r"(.*?)(?:\[.*?\]|$)", data)
                log_text = out_brackets[2].strip().split(" ")
                
                time = dt.datetime.strptime(in_brackets[1], DATETIME_FORMAT)
                
                overflow_bus_id = dispatch_info.split(" ")[-3].split("'")[1]
                broken_bus = dispatch_info.split(" ")[-1].split("'")[1]
                
                trip_id = df.query(f"bus_id == '{broken_bus}'").iloc[-1].trip_id
                stop_id = log_text[-1]
                from_stop = log_text[-4]
                
                entry = {"actual_time":time,
                        "bus_id":overflow_bus_id,
                        "from_stop":from_stop,
                        "trip_id":trip_id,
                        "stop_id":stop_id,
                        "type":"broken"}
                bus_stats.append(entry)
                lines_before = []
    bus_stats = pd.DataFrame(bus_stats)
    bus_stats = bus_stats.drop_duplicates()
    return bus_stats

dispatch_arr = []
for log_file in log_files:
    dispatch_df = get_dispatch_df(log_file)
    dispatch_arr.append(dispatch_df)

In [515]:
# Find other trips of bus 1829
# "242142"
# "242177"
# "242177"
# "242147"

# "242181"
        
# "241652"
# "241678"
# "242151"
# "242185"
# "241656"
# "241680"
# "242155"
# "242189"
# "241660"
# "241682"
# "242159"
# "242193"
# "241664"
# "241684"
# "242163"
# "242197"
# "241668"
# "241686"
# "242167"
# "242201"
# "241670"
# "241688"
# "242171"
# "242205"
# "241672"
# "241690"
# "243471"
# "243465"
# "243516"
      
df.query("bus_id == '45' and trip_id == '242181'")

,actual_time,bus_id,trip_id,scheduled_time,stop_id,got_on_bus,waiting_to_board,offs,remain,load,overload_id
11416,2021-06-07 09:14:24,45,242181,2021-06-07 08:59:00,JEF8AVEF,0,0,0,0,1,1
11422,2021-06-07 09:14:32,45,242181,2021-06-07 09:00:02,JEF6AVEN,3,3,0,0,4,1
11432,2021-06-07 09:14:50,45,242181,2021-06-07 09:01:42,5AVJRGSN,0,0,1,0,3,1
11464,2021-06-07 09:15:13,45,242181,2021-06-07 09:03:46,5AVHARSN,2,2,0,0,5,1
11485,2021-06-07 09:15:44,45,242181,2021-06-07 09:07:00,MCC5_11,0,0,5,0,0,1


In [516]:
dispatch_arr[1].set_index('actual_time').between_time('8:00', '9:20')
# 245790 245787 245855 244871 243379

,bus_id,from_stop,trip_id,stop_id,type
actual_time,,,,,
2021-06-07 08:03:21,44,100OAKS,243264,MAYDUPSN,overflow
2021-06-07 08:05:00,43,MCC5_4,245784,UNI2AWN,overflow
2021-06-07 08:06:22,43,MCC4_24,242146,33AJOHSN,overflow
2021-06-07 08:09:52,41,MCC5_6,243518,GRAFERSN,overflow
2021-06-07 08:11:04,42,MCSHERM,243177,LEBOLDNN,overflow
2021-06-07 08:12:12,43,33AJOHNM,245281,H70ERIWN,overflow
2021-06-07 08:18:35,41,LPSCI,246302,UNI7AEN,overflow
2021-06-07 08:23:00,42,MCSHERM,244794,NXOPBODY,overflow
2021-06-07 08:28:28,41,MCC4_23,244871,MURBORNF,overflow


In [517]:
fig = make_subplots(
    rows=3, cols=1,
    horizontal_spacing = 0.05, vertical_spacing=0.1
)
trip_ids = ["242177", "242147", "242181", "241652", "241678", "242151", "242185"]# + ["245790", "245787", "245855", "244871", "243379"]

for df_index in [0, 1, 2]:
  tdf = df_arr[df_index]
  tdf = tdf[tdf['trip_id'].isin(trip_ids)]
  dt_indices = df_arr[df_index].actual_time.unique().tolist() + dispatch_arr[df_index].actual_time.unique().tolist()
  all_indices = merge_all_indices(tdf, dt_indices)

  only_valid = all_indices[all_indices['trip_id'].isin(trip_ids)]
  min_time = (only_valid.index.min() - dt.timedelta(minutes=5)).time()
  max_time = (only_valid.index.max() - dt.timedelta(minutes=5)).time()
  merged_df = all_indices.between_time(min_time, max_time)
  merged_df = merged_df[merged_df['trip_id'].isin(trip_ids)]
  merged_df['zero'] = 0
  merged_df['remain'] = merged_df['remain'] * -1
  
  fig.add_trace(go.Scatter(x=merged_df.index, y=merged_df.zero, line=dict(color='black', width=1), showlegend=False), row=df_index+1, col=1)

  # marker_pattern_shape=".", marker_pattern_fillmode='replace'
  traces = [go.Bar(x=trip_df.index, y=trip_df.got_on_bus, width=25000, name="regular bus passenger served", marker_color='grey', marker_opacity=0.5, showlegend=(i==0 and df_index == 0)) \
              for i, (trip_id, trip_df) in enumerate(merged_df.query("trip_id != 0 and overload_id == 0").groupby("trip_id"))]
  fig.add_traces(traces, rows=df_index+1, cols=1)

  fig.add_vrect(x0=merged_df.index.min(), x1=merged_df.index.max(), 
            annotation_text=f"Total served:{int(merged_df.got_on_bus.sum())}", 
            annotation_position="top left", annotation_textangle=0, annotation=dict(font=dict(size=20), bgcolor="rgba(255,255,255,0.8)"),
            fillcolor="green", opacity=0, line_width=0, row=df_index+1, col=1)
  
  marker_colors = ['blue', 'blue']
  traces = [go.Bar(x=trip_df.index, y=trip_df.got_on_bus, width=35000, name="overflow bus passengers served", marker_color=marker_colors[i%2], showlegend=(i==0 and df_index == 0)) \
              for i, (trip_id, trip_df) in enumerate(merged_df.query("trip_id != 0 and overload_id == 1").groupby("trip_id"))]
  fig.add_traces(traces, rows=df_index+1, cols=1)

  dispatch_df = dispatch_arr[df_index]
  dispatch_df = dispatch_df[dispatch_df['trip_id'].isin(trip_ids)]
  all_indices = merge_all_indices(dispatch_df, dt_indices).query("bus_id != 0")
  display(all_indices)
  for i, (k, v) in enumerate(all_indices.sort_index().iterrows()):
      fig.add_trace(go.Scatter(x=[k,k], 
                          y=[0, 0], 
                          mode='lines', 
                          line=dict(color='black', dash='dash'),
                          name='overflow dispatched', showlegend=(i==0 and df_index == 0)), row=df_index+1, col=1)
      
      trip_id = v['trip_id']
      bus_id = v['bus_id']
      fig.add_vline(x=k, line_width=2, line_dash="dash", line_color="black", row=df_index+1, col=1)
      a = merged_df.query(f"bus_id == '{bus_id}'")
      # a = merged_df.query(f"bus_id == '{bus_id}' and trip_id == '{trip_id}'")
      a_start = a.index.min()
      a_end = a.index.max()
      a_sum_served = int(a.got_on_bus.sum())
      # a_start, a_end, a_sum_served
      if i % 2 == 0:
        shift = -80
      else:
        shift = 0
      # print(a_sum_served, i, shift)
      fig.add_vrect(x0=k, x1=a_end, 
              #   annotation_text=f"Total served:{a_sum_served}", annotation_position="top left",
                annotation_text=f"{bus_id}:{a_sum_served}", annotation_position="top left", annotation_textangle=0, annotation=dict(font=dict(size=20), yshift=shift, bgcolor="rgba(255,255,255,0.8)"),
                fillcolor="green", opacity=0, line_width=0, row=df_index+1, col=1)

  if df_index > 0:
    fig.add_vline(x='2021-06-07 09:00:00', line_width=2, line_dash="solid", line_color="red", row=df_index+1, col=1)
  
  fig.add_trace(go.Scatter(x=['2021-06-07 09:00:00', '2021-06-07 09:00:00'], 
                      y=[0, 0], 
                      mode='lines', 
                      line=dict(color='red', dash='solid'),
                      name='disruption event', showlegend=(df_index == 0)), row=df_index+1, col=1)
    
  fig.update_yaxes(range=[0, 50])

# fig.update_layout(showlegend=False)
fig.update_layout(margin={"r":10,"t":10,"l":10,"b":10}, width=1400, height=600)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
    font=dict(size=15)
))
# fig.update_yaxes(range=[-40, 40], title='passengers served')

# fig.write_image("plots/timeseries_comparison_breakdowns.eps")
fig.show()

,bus_id,from_stop,trip_id,stop_id,type
2021-06-07 10:29:00,42,HICHICNN,242151,JEF9AWN,overflow


,bus_id,from_stop,trip_id,stop_id,type
2021-06-07 09:04:19,44,MCC5_6,242181,JEF8AVEF,broken


,bus_id,from_stop,trip_id,stop_id,type
2021-06-07 08:27:24,42,WALMARTC,242147,MCC5_8,overflow
2021-06-07 09:12:13,45,DOMFREEN,242181,JEF8AVEF,broken


In [508]:
adf = df_arr[1].query("bus_id == '45' and actual_time > '2021-06-07 09:14:24'")
with pd.option_context('display.max_rows', 5,):
    display(adf)

,actual_time,bus_id,trip_id,scheduled_time,stop_id,got_on_bus,waiting_to_board,offs,remain,load,overload_id
10883,2021-06-07 09:15:07,45,244796,2021-06-07 09:15:07,MXOZELID,0,0,1,0,54,1
10964,2021-06-07 09:17:06,45,244796,2021-06-07 09:17:06,MXOMTVIE,0,0,0,0,54,1
...,...,...,...,...,...,...,...,...,...,...,...
42948,2021-06-08 00:31:29,45,243468,2021-06-07 23:10:25,JAM5AEN,0,0,0,0,0,1
42949,2021-06-08 00:31:48,45,243468,2021-06-07 23:12:00,MCC5_4,0,0,0,0,0,1
